In [1]:
import pandas as pd
import numpy as np
from nltk import tag, word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [2]:
## load data
df = pd.read_csv('data/S10/question_answer_pairs.txt', sep='\\t', engine='python').dropna()

questions = df['Question']
answers = df['Answer']

In [14]:
## penn-tree tokenization
def tokenizer(sentence):
    # penn-tree: tokenize + parts of speech
    sent = word_tokenize(sentence)
    pos = tag.pos_tag(sent)
    return([x[1] for x in pos if x[1]])

In [15]:
##
## order of appending 'pos' (1) must match the order of appending
##     the 'sent_type' (2) sentence type.
##

## (1) pos: combine questions + answers
questions_pos = questions.apply(tokenizer)
questions_unique = [list(x) for x in set(tuple(x) for x in questions_pos)]

answers_pos = answers.apply(tokenizer)
answers_unique = [list(x) for x in set(tuple(x) for x in answers_pos)]

## append data
pos = answers_pos.append(questions_pos)

## (2) sentence type
sent_type = []
for i in range(len(questions_pos)):
    sent_type.append('0')

for j in range(len(answers_pos)):
    sent_type.append('1')

In [16]:
## adjusted dataset
df_adjusted = pd.DataFrame({
    'pos': pos,
    'type': sent_type
})

## train + test
X_train, X_test, y_train, y_test = train_test_split(
    df_adjusted['pos'],
    df_adjusted['type'],
    test_size=0.2
)

## print shape
print('X_train: {X}, y_train: {y}'.format(
    X=X_train.shape,
    y=y_train.shape
))

print('X_test: {X}, y_test: {y}'.format(
    X=X_test.shape,
    y=y_test.shape
))

X_train: (1641,), y_train: (1641,)
X_test: (411,), y_test: (411,)


In [17]:
df_adjusted.head(10)

,pos,type
0,"[NNP_Alessandro, NNP_Volta, VBD_was, RB_not, D...",0
1,[DT_No],0
2,"[NNP_Alessandro, NNP_Volta, VBD_did, VB_invent...",0
3,[UH_Yes],0
4,"[NNP_Volta, VBD_was, VBN_taught, IN_in, JJ_pub...",0
5,[UH_Yes],0
6,"[NNP_Alessandro, NNP_Volta, VBD_married, NNP_T...",0
7,"[NNP_Teresa, NNP_Peregrini]",0
8,"[IN_In, CD_1800, ,_,, NNP_Alessandro, NNP_Volt...",0
9,"[NNS_voltaic, NN_pile]",0


In [27]:
## ensure nested lists same length
length = len(sorted(X_train, key=len, reverse=True)[0])
y=np.array([xi+[None]*(length-len(xi)) for xi in X_train])
y=pd.DataFrame(y)

In [28]:
##
## random forrest: selected due to high accuracy, and hasn't
##     been implemented in the project.
##
clf=RandomForestClassifier(n_estimators=5)
clf.fit(y, np.asarray(y_train))
#y_pred=clf.predict(X_test)

ValueError: could not convert string to float: '._?'